In [1]:
#Importing all the dependencies

import pandas as pd
import itertools
import matplotlib.pyplot as plt
plt.style.use('default')
import seaborn as sns
import numpy as np
import re
import string
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.corpus import stopwords 
from textblob import Word, TextBlob
from wordcloud import WordCloud

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


ImportError: cannot import name 'triu' from 'scipy.linalg' (/home/codespace/.local/lib/python3.10/site-packages/scipy/linalg/__init__.py)

In [ ]:
df = pd.read_csv('Datasets/fake_or_real_news_with_sentiment.csv')

In [ ]:
df.head()

,Unnamed: 0,title,text,label,sentiment
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [ ]:
df.shape

(6335, 5)

In [ ]:
df['sentiment'].value_counts()

sentiment
1    5786
5     269
2     157
3      84
4      39
Name: count, dtype: int64

In [ ]:
df=df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df['ID'] = range(1, len(df) + 1) #This creates a column titled ID, that starts at 1 and is the length of the DataFrame. 
first_column = df.pop('ID')
df.insert(0, 'Id', first_column) #This makes the newly created column the first in the dataframe.

In [ ]:
#Removes null values
df.isnull().sum()

Id           0
title        0
text         0
label        0
sentiment    0
dtype: int64

In [ ]:
df['text'] = df['text'].str.lower().str.replace('\\s+', ' ', regex=True).str.strip()

In [ ]:
#Preprocessing Text
def preprocessedTexts(row):
    text = row['text']
    text = p.clean(text)
    return  text
df['PreprocessedTexts'] = df.apply(preprocessedTexts, axis=1)
df.head()

AttributeError: module 'preprocessor' has no attribute 'clean'

In [ ]:
tokens = nltk.word_tokenize('text')

NameError: name 'text' is not defined

In [ ]:
from nltk.corpus import stopwords

stop_words=set(stopwords.words("english"))
print(stop_words)

In [ ]:
tags = nltk.pos_tag(tokens)

Vader Model 

In [ ]:
# Get Polarity Scores
#Empty dictionary
results = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['PreprocessedTexts']
    rowID = row['ID']
    results[rowID] = sia.polarity_scores(text)
pd.DataFrame(results).T 

In [ ]:
pd.DataFrame(results).T.describe()

In [ ]:
#Merge the polarity scores data frame with the main data frame
vader = pd.DataFrame(results).T
vader = vader.reset_index().rename(columns={'index': 'ID'})
vader = vader.merge(df, how='left')
vader

In [ ]:
#This is to compute whether it is Negative, Neutral or Positive
def GetAnalysis(score):
    if score <0:
        return "Negative"
    elif score==0:
        return "Neutral"
    else:
        return "Positive"
vader['VaderAnalysis']=vader['compound'].apply(GetAnalysis)
vader['VaderAnalysis'].head()

In [ ]:
#To calculate the percentage of 'Positive' texts
positiveTexts = vader[vader.VaderAnalysis=='Positive']
print(round((positiveTexts.shape[0]/vader.shape[0])*100,1))

In [ ]:
#To calculate the percentage of 'Negative' texts
negativeTexts = vader[vader.VaderAnalysis=='Negative']
print(round((negativeTexts.shape[0]/vader.shape[0])*100,1))

In [ ]:
#To calculate the percentage of 'Netrual' texts
neutralTexts = vader[vader.VaderAnalysis=='Netrual']
print(round((neutralTexts.shape[0]/vader.shape[0])*100,1))

In [ ]:
#Print Positive Texts
j=1
sortedDataFrame= vader.sort_values(by=["compound"])
for i in range(0,sortedDataFrame.shape[0]):
    if (sortedDataFrame['VaderAnalysis'][i]=='Positive'):
        print(str(j)+'.'+sortedDataFrame['PreprocessedTexts'][i])
        print()
        j=j+1

In [ ]:
#Print Negative Texts
j=1
sortedDataFrame= vader.sort_values(by=["compound"])
for i in range(0,sortedDataFrame.shape[0]):
    if (sortedDataFrame['VaderAnalysis'][i]=='Negative'):
        print(str(j)+'.'+sortedDataFrame['PreprocessedTexts'][i])
        print()
        j=j+1

In [ ]:
#Print Neutral Texts
j=1
sortedDataFrame= vader.sort_values(by=["compound"])
for i in range(0,sortedDataFrame.shape[0]):
    if (sortedDataFrame['VaderAnalysis'][i]=='Neutral'):
        print(str(j)+'.'+sortedDataFrame['PreprocessedTexts'][i])
        print()
        j=j+1

In [ ]:
X_train , X_test, y_train, y_test = train_test_split(vader).final.drop(['ID'], axis=1), vader.Label, test_size=0.2, random_state
vm = LinearDiscriminantAnalysis().fit(X_train,y_train)
predictions = vm.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
columns = ['ID', 'neg', 'neu','pos','text','sentiment',]
columns_final = df[columns]
X_train , X_test, y_train, y_test = train_test_split(vader).final.drop(['ID'], axis=1), columns_final.Label, test_size=0.2, random_state
vm = LinearDiscriminantAnalysis().fit(X_train, y_train)
predictions = vm.predict(X_test)
print(classification_report(y_test,predictions))

Roberta Model

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
#Pretrained model by huggingface
roBERTa = f"cardiffnlp/twitter-roberta-base-sentiment" #PretrainedModel
tokenizer = AutoTokenizer.from_pretrained(roBERTa)
rb = AutoModelForSequenceClassification.from_pretrained(roBERTa)
roberta_pipeline = pipeline("sentiment-analysis", model=rb, tokenizer=tokenizer)

# Initialize VADER
sia = SentimentIntensityAnalyzer()

In [ ]:
#example
example='He does not like vegetables'
text_encoded=tokenizer(example,return_tensors='pt') #Eencodes the text into 1s and 0s for the model to understand
output = rb(**text_encoded)
scores = output[0][0].detach().numpy()#Roberta gives scores as tensors. this changes it to numpy. 
scores = softmax(scores) #Apply softmax to give the values between 0 and 1
scoresResult = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scoresResult)

In [ ]:
#Create a function out if the code we ran on the example above to then apply to DataFrame after
def polarityRoBERTa(example):
    text_encoded=tokenizer(example,return_tensors='pt')
    output = rb(**text_encoded)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scoresResults = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scoresResults

In [ ]:
# Run the polarity score on the entire dataset
results = {}
for i, row in tqdm(vader.iterrows(), total=len(df)):
    try:
        texts = row['PreprocessedTexts']
        rowID = row['ID']
        vader_results = sia.polarity_scores(texts)
        vader_result_rename = {}

        for key, value in vader_results.items():
            vader_result_rename[f"vader_{key}"] = value
            roberta_results=polarityRoBERTa(text)
            both = {**vader_result_rename, **roberta_results}
            results [rowID] = both

    except RuntimeError:
        print(f'broke for id{rowID}')


#Merge bot of the DataFrames into new DataFrame called resultsDF
resultsDF = pd.DataFrame(results ).T
resultsDF = resultsDF.reset_index().rename(columns={'index': 'Id'})
resultsDF = resultsDF.merge(vader, how='left')
resultsDF = resultsDF.drop(['neg','pos','neu','compound'],axis=1) #Remove this from DataFrame

In [ ]:
resultsDF.head()


In [ ]:
#Use lambda function to label tweet as positive, negative or neutral
resultsDF['Rob_Analysis'] = resultsDF.apply(lambda x:'Negative' if x['roberta_neg'] >
                     x['roberta_neu'] and x['roberta_neu']
                     > x['roberta_pos'] else ('Neutral' if x['roberta_neu']>x['roberta_pos'] else 'Positive'),axis=1)
resultsDF['Rob_Analysis'].head()

RoBERTa does not compute the compound score so here we have to compare the 3 probabilities and the Highest Score becomes the label of the text.

In [ ]:
#Percentage of Positive Texts
positiveTexts=resultsDF[resultsDF.Rob_Analysis=='Positive']
print(round((positiveTexts.shape[0]/resultsDF.shape[0])*100,1))

In [ ]:
#percentage of Negative Texts
negativeTexts=resultsDF[resultsDF.Rob_Analysis=='Negative']
print(round((negativeTexts.shape[0]/resultsDF.shape[0])*100,1))

In [ ]:
#percentage of Negative Texts
neutralTexts=resultsDF[resultsDF.Rob_Analysis=='Neutral']
print(round((neutralTexts.shape[0]/resultsDF.shape[0])*100,1))

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
sns.set(style="white")
ax = sns.countplot(y="VaderAnalysis",  data=resultsDF)

plt.figure(figsize=(14,4))
plt.subplot(1,2,2)
sns.set(style="white")
ax = sns.countplot(y="Rob_Analysis",  data=resultsDF)

In [ ]:
#Print Positive texts
j=1
sortedDF=resultsDF.sort_values(by=["ID"])
for i in range(0,sortedDF.shape[0]):
    if (sortedDF['Rob_Analysis'][i]=='Positive'):
        print(str(j)+'.'+sortedDF['PreprocessedTexts'][i])
        print()
        j=j+1

In [ ]:
#Print Negative texts
j=1
sortedDF=resultsDF.sort_values(by=["ID"])
for i in range(0,sortedDF.shape[0]):
    if (sortedDF['Rob_Analysis'][i]=='Negative'):
        print(str(j)+'.'+sortedDF['PreprocessedTexts'][i])
        print()
        j=j+1

In [ ]:
#Printing Neutral texts
j=1
sortedDF=resultsDF.sort_values(by=["ID"])
for i in range(0,sortedDF.shape[0]):
    if (sortedDF['Rob_Analysis'][i]=='Neutral'):
        print(str(j)+'.'+sortedDF['PreprocessedTexts'][i])
        print()
        j=j+1

In [ ]:
def combined_sentiment_analysis(text, weight_roberta=0.6, weight_vader=0.4):

    # Getting  sentiment from RoBERTa
    roberta_result = roberta_pipeline(text)[0]
    roberta_sentiment = roberta_result['label']
    roberta_confidence = roberta_result['score']
    
    # Convert RoBERTa sentiment to a unified score
    roberta_score = 1 if roberta_sentiment == 'POSITIVE' else (0 if roberta_sentiment == 'NEUTRAL' else -1)

    # Get sentiment from VADER
    vader_result = sia.polarity_scores(text)
    vader_score = vader_result['compound']  # Using compound score 

    # Calculate combined score
    combined_score = (weight_roberta * roberta_score * roberta_confidence) + (weight_vader * vader_score)
    
    # Return combined sentiment
    return "POSITIVE" if combined_score > 0 else ("NEGATIVE" if combined_score < 0 else "NEUTRAL")

combined_sentiment = combined_sentiment_analysis(sortedDF)
print("Combined Sentiment:", combined_sentiment)

In [ ]:
vm = LinearDiscriminantAnalysis()

In [ ]:
rb = AutoModelForSequenceClassification.from_pretrained(roBERTa)

In [ ]:
Combined_Models = VotingClassifier(estimators=[('vm', vm), ('rb', rb)], voting='soft')

# Fit our training data to the ensemble model

Combined_Models = Combined_Models.fit(x_train, y_train)

In [ ]:
#predict labels for testing set 
y_pred_combined = Combined_Models.predict(X_test_test)

In [ ]:
accuracy_combined = accuracy_score(y_test, y_pred_combined)
print("Combined Accuracy: ", accuracy_combined)